In [2]:
# Use the best model for each checkpoint folder to test maes for all data folds

In [3]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
from pathlib import Path
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [4]:
args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/Label'  #real label
# args.output = '/project/results/soutput1/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput2/snapshots/'
# args.evalpath = '/project/results/sroutput1/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=35
args.angle = 180
args.bin_width = 4


In [5]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
# evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width
gazeMpiimage_dir = args.gazeMpiimage_dir
gazeMpiilabel_dir=args.gazeMpiilabel_dir

In [6]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used= getArch(arch, bins)  #resnet50 and 28 bins

In [7]:
#read the fold epoch table
dfile = Path(snapshot_path).parent/"syn_syn_sdata2_all_MAE.npy"
all_MAE = np.load(dfile)
best_epoch = {}
best_epoch_error = {}
print(f"fold   best_epoch error(degree)")
for idx, fdata in enumerate(all_MAE):
#     print(idx, type(idx))
    epoch_min_error = np.argmin(fdata)+1
    print(f'{idx:^4} {np.argmin(fdata)+1:^12} {fdata.min():^11.3f}')
    best_epoch[idx+1] = epoch_min_error
    best_epoch_error[idx+1]=fdata.min()

fold   best_epoch error(degree)
 0        47         2.902   
 1        55         2.718   
 2        56         3.145   
 3        57         3.704   
 4        52         3.123   
 5        51         2.578   
 6        58         3.239   
 7        55         2.979   
 8        47         4.673   
 9        32         4.767   
 10       26         2.979   
 11       53         2.528   
 12       10         3.276   
 13       54         2.488   
 14       11         3.323   


In [8]:
# check if we have the correct number of checkpoint files 
ppath = snapshot_path
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(f'{fold}:{len(files)}',end=" ")

0:60 1:60 2:60 3:60 4:60 5:60 6:60 7:60 8:60 9:60 10:60 11:60 12:60 13:60 14:60 

In [9]:
def get_now():
    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format) 
    return now
print(get_now())

07/14/2022 08:08:50


In [10]:
%%time
all_MAE = []
# tosave={}
lfolder = os.listdir(gazeMpiilabel_dir)
lfolder.sort()  #individual label files
testlabelpathcombined = [os.path.join(gazeMpiilabel_dir, j) for j in lfolder] 
print(testlabelpathcombined)
model = model_used
model= nn.DataParallel(model,device_ids=[0])  #important to load state dict
for cfold in range(15):  #checkpoint folds
    frun = wandb.init(project="63_best_checkpoint_model_all_real_data_folds")
    frun.name = f'checkpoint fold{cfold}_mae'
    
    cfoldstr = f"fold{cfold:0>2}"
    print(f"cfold={cfoldstr}")
    idx = cfold + 1
    b_epoch = best_epoch[idx]
    
    checkpoint_path = Path(snapshot_path)/cfoldstr
    best_checkpoint_file = 'epoch_'+str(b_epoch)+'.pkl'
    best_checkpoint_file = checkpoint_path/best_checkpoint_file
#     print(best_checkpoint_file, type(best_checkpoint_file))
    checkpoint = torch.load(best_checkpoint_file)
    saved_state_dict = checkpoint['model_state_dict']
    model.load_state_dict(saved_state_dict)

    model.cuda(gpu)
    model.eval()
    idx_tensor = [idx for idx in range(35)]
    idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
    softmax = nn.Softmax(dim=1)
    best_model_maes =[]  # error for each data fold
    for dfold in range(15):  #data folds
        gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, gazeMpiimage_dir, transformations, False, angle, dfold)
        test_loader = torch.utils.data.DataLoader(
            dataset=gaze_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=True)
        now = get_now()
        configuration = f"\ntest configuration: gpu_id={gpu}, batch_size={batch_size}\n"
        configuration += f"model_arch={arch} Start testing dataset={data_set}\n"
        configuration +=  f"cFOLD={cfold} dFOLD={dfold}--{now}----"
        print(configuration)
        
        total = 0  
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 52
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 52

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
                    pl, yl = yl, pl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl])) #accumulate over all batches
                    
        mean_mae = avg_error/total  
        now = get_now()
        msg = f"[cfold{cfold}---dfold {dfold}] Total Num:{total}, MAE:{mean_mae}  {now}"
#         outfile.write(loger)
        print(msg)
        best_model_maes.append(mean_mae)  # a row of maes for each data fold
        # log the mean error for the data fold for diagnostic purposes
        frun.log({'data fold': dfold, f'dfold_{dfold}_avg_mae':mean_mae}, step=dfold)
    bestmodel_mean = np.mean(best_model_maes)  # the mean mae for the model  
    msg = f"[cfold{cfold}--] MAE for this fold best model:{bestmodel_mean}  {now}"
    print(msg)
    all_MAE.append(best_model_maes)  # append the whole row of data fold maes for each best fold model
    # log for each best model
    

['/project/data/Label/p00.label', '/project/data/Label/p01.label', '/project/data/Label/p02.label', '/project/data/Label/p03.label', '/project/data/Label/p04.label', '/project/data/Label/p05.label', '/project/data/Label/p06.label', '/project/data/Label/p07.label', '/project/data/Label/p08.label', '/project/data/Label/p09.label', '/project/data/Label/p10.label', '/project/data/Label/p11.label', '/project/data/Label/p12.label', '/project/data/Label/p13.label', '/project/data/Label/p14.label']


wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


cfold=fold00
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=0 dFOLD=0--07/14/2022 08:08:57----
[cfold0---dfold 0] Total Num:3000, MAE:4.760603778014664  07/14/2022 08:09:12
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=0 dFOLD=1--07/14/2022 08:09:12----
[cfold0---dfold 1] Total Num:3000, MAE:5.961119076162659  07/14/2022 08:09:25
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=0 dFOLD=2--07/14/2022 08:09:25----
[cfold0---dfold 2] Total Num:3000, MAE:5.588073937308223  07/14/2022 08:09:38
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=0 dF

data fold,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
dfold_0_avg_mae,▁
dfold_10_avg_mae,▁
dfold_11_avg_mae,▁
dfold_12_avg_mae,▁
dfold_13_avg_mae,▁
dfold_14_avg_mae,▁
dfold_1_avg_mae,▁
dfold_2_avg_mae,▁
dfold_3_avg_mae,▁
dfold_4_avg_mae,▁


cfold=fold01
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=1 dFOLD=0--07/14/2022 08:12:22----
[cfold1---dfold 0] Total Num:3000, MAE:5.034936656205768  07/14/2022 08:12:35
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=1 dFOLD=1--07/14/2022 08:12:35----
[cfold1---dfold 1] Total Num:3000, MAE:7.096040153084886  07/14/2022 08:12:48
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=1 dFOLD=2--07/14/2022 08:12:48----
[cfold1---dfold 2] Total Num:3000, MAE:5.589969066774633  07/14/2022 08:13:01
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=1 dF

...

cfold=fold12
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=12 dFOLD=0--07/14/2022 08:49:28----
[cfold12---dfold 0] Total Num:3000, MAE:4.857964054047415  07/14/2022 08:49:41
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=12 dFOLD=1--07/14/2022 08:49:41----
[cfold12---dfold 1] Total Num:3000, MAE:9.574976318326314  07/14/2022 08:49:54
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=12 dFOLD=2--07/14/2022 08:49:54----
[cfold12---dfold 2] Total Num:3000, MAE:6.0093045545482155  07/14/2022 08:50:07
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFO

data fold,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
dfold_0_avg_mae,▁
dfold_10_avg_mae,▁
dfold_11_avg_mae,▁
dfold_12_avg_mae,▁
dfold_13_avg_mae,▁
dfold_14_avg_mae,▁
dfold_1_avg_mae,▁
dfold_2_avg_mae,▁
dfold_3_avg_mae,▁
dfold_4_avg_mae,▁


cfold=fold13
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=13 dFOLD=0--07/14/2022 08:52:50----
[cfold13---dfold 0] Total Num:3000, MAE:5.65568337995294  07/14/2022 08:53:03
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=13 dFOLD=1--07/14/2022 08:53:03----
[cfold13---dfold 1] Total Num:3000, MAE:7.7442026083804505  07/14/2022 08:53:16
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=13 dFOLD=2--07/14/2022 08:53:16----
[cfold13---dfold 2] Total Num:3000, MAE:5.709074445495728  07/14/2022 08:53:29
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOL

data fold,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
dfold_0_avg_mae,▁
dfold_10_avg_mae,▁
dfold_11_avg_mae,▁
dfold_12_avg_mae,▁
dfold_13_avg_mae,▁
dfold_14_avg_mae,▁
dfold_1_avg_mae,▁
dfold_2_avg_mae,▁
dfold_3_avg_mae,▁
dfold_4_avg_mae,▁


cfold=fold14
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=14 dFOLD=0--07/14/2022 08:56:13----
[cfold14---dfold 0] Total Num:3000, MAE:4.849690487373123  07/14/2022 08:56:26
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=14 dFOLD=1--07/14/2022 08:56:26----
[cfold14---dfold 1] Total Num:3000, MAE:7.175460665725654  07/14/2022 08:56:39
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOLD=14 dFOLD=2--07/14/2022 08:56:39----
[cfold14---dfold 2] Total Num:3000, MAE:7.294637746643371  07/14/2022 08:56:51
0 items removed from dataset that have an angle > 180

test configuration: gpu_id=cuda:0, batch_size=20
model_arch=ResNet50 Start testing dataset=mpiigaze
cFOL

In [10]:
all_MAE=np.array(all_MAE)

In [11]:
print(all_MAE.shape)
print("data fold means, which data fold is bad or good:\n",all_MAE.mean(axis=0))
print("model means, which model ot choose:\n", all_MAE.mean(axis=1))
# print("data fold min:", all_MAE.min(axis=0))
# print("model min:", all_MAE.min(axis=1))
# print("data fold max:", all_MAE.max(axis=0))
# print("model max:", all_MAE.max(axis=1))

(15, 15)
data fold means, which data fold is bad or good:
 [ 5.0663695   7.35310036  5.77627721  7.54881561  5.80370927  6.16096575
 10.9717504   7.61653907  7.42706456  8.13063561 10.19430371  7.70687702
  8.38090911  9.08737908  6.18312436]
model means, which model ot choose:
 [7.27013484 7.30106072 6.97990799 6.78448713 7.5874587  7.24025782
 7.78974593 6.81804498 7.14187596 9.12686916 7.34980803 7.21050472
 8.43433659 8.11510012 8.25822796]


In [12]:
save_file = Path(snapshot_path).parent/"cfold_dfold_sdata2_all_MAE.npy"
with open(save_file, 'wb') as f:
    np.save(f, all_MAE)